# Intro to Pandas

### Jan 25th, 2019

Topics covered : 
* Series & DataFrames
* Basic Summary methods
* Selecting & Filtering Data
* Creating New Variables
* Groupby operations
* Merge operations
* Reading Files
* Class Exercises


In [ ]:
import pandas as pd
import numpy as np

# !pip install jupyter_contib_nbextensions

# Topics

## Series & DataFrames

In [ ]:
sports = pd.Series(['football', 'basketball',' volleyball','tennis'])

population = pd.Series({'Germany': 81.3, 'Belgium': 11.3, 'France': 64.3, 
                        'United Kingdom': 64.9, 'Netherlands': 16.9})

countries = pd.DataFrame({'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9],
        'area': [30510, 671308, 357050, 41526, 244820],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']})

In [ ]:
sports

In [ ]:
population

In [ ]:
countries

In [ ]:
type(population)

In [ ]:
sports.index

In [ ]:
population.index

In [ ]:
population['Belgium']

In [ ]:
population.values

In [ ]:
population/100

In [ ]:
type(population.values)

In [ ]:
type(countries)

In [ ]:
countries

Accessing dataframe variables using the '.' operator

In [ ]:
type(countries.area)

In [ ]:
countries.area.values

In [ ]:
type(countries.capital.values)

## Basic Methods

In [ ]:
countries.columns

In [ ]:
countries.dtypes

In [ ]:
countries.head()

In [ ]:
countries.describe()

In [ ]:
countries.values

In [ ]:
countries.info()

In [ ]:
countries.capital.value_counts()

In [ ]:
population

In [ ]:
population.reset_index()

In [ ]:
type(population.reset_index())

In [ ]:
countries.capital.value_counts().reset_index()

## Selecting and Filtering Data

<div class="alert alert-warning">
<b>ATTENTION!</b>: <br><br>

One of pandas' basic features is the labeling of rows and columns, but this makes indexing also a bit more complex compared to numpy. <br><br> We now have to distuinguish between:

 <ul>
  <li>selection by **label**</li>
  <li>selection by **position**</li>
</ul>
</div>

In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
df.head()

### `data[]` provides some convenience shortcuts 

Selecting a single column

In [ ]:
df['Pclass']  # Can also use df.Pclass

Selecting multiple columns

In [ ]:
df[['Pclass','Sex']]

Keep in mind that when we select more than one column, the output is DataFrame and not a series. Hence the difference in formatting of the two outputs above





We can also use this syntax to select specific rows

In [ ]:
df[3:5]

### Systematic indexing with `loc` and `iloc`

When using `[]` like above, you can only select from one axis at once (rows or columns, not both). For more advanced indexing, you have some extra attributes:
    
* `loc`: selection by label
* `iloc`: selection by position

These methods index the different dimensions of the frame:

* `df.loc[row_indexer, column_indexer]`
* `df.iloc[row_indexer, column_indexer]`

In [ ]:
df.loc[4,'Fare']

In [ ]:
df.loc[df.Sex=='female']

In [ ]:
df.loc[df.Sex=='female','Fare']

In [ ]:
df.loc[df.Sex=='female',['Fare','Name','Sex']]

In [ ]:
df.loc[df.Sex=='female'][['Fare','Name','Sex']]

iloc is based on the position of the elements

In [ ]:
df.iloc[4]

In [ ]:
df.iloc[5:7]

In [ ]:
df.iloc[5:7,'Fare']

In [ ]:
df.iloc[5:7]['Fare']

In [ ]:
df.iloc[[1,2,3,4]]

The different indexing methods can also be used to assign data:

In [ ]:
df2 = df.copy()

df2.loc[0,'Fare'] = -100.0

In [ ]:
df2.head()

Rows can be selected based on whether or not they satisfy a certain (boolean) condition

In [ ]:
df[df.Fare>100]

## Creating New Variables

In [ ]:
countries['newVar'] = [1,2,3,4,5]                   #Basic assignment
countries

In [ ]:
countries['newVar'] = countries.population * 2  + countries.area**0.5   #Using existing columns
countries

### Using apply

Apply is a very powerful method which can be used for making major data manipulation tasks

In [ ]:
countries['CAPITAL'] = countries['capital'].apply(lambda x : x.upper())
countries

In [ ]:
def ageBucket(x):
    if x<18:
        return "A. <18"
    elif x<25:
        return "B. 18-25"
    elif x<45:
        return "C. 25-45"
    else:
        return "D. >45"
        

Apply can be used on a single column (Series object)

In [ ]:
df['AgeBucket'] = df['Age'].apply(lambda x : ageBucket(x))
df.head()

It can also be used on an entire dataframe

In [ ]:
df['AgeBucket2'] = df.apply(lambda x : ageBucket(x['Age']),axis=1)
df.head()

Other derivative methods that you can look into : `map` and `applymap`

## Groupby Operations

### Some 'theory': the groupby operation (split-apply-combine)

The "group by" concept: we want to **apply the same function on subsets of your dataframe, based on some key to split the dataframe in subsets**

This operation is also referred to as the "split-apply-combine" operation, involving the following steps:

* **Splitting** the data into groups based on some criteria
* **Applying** a function to each group independently
* **Combining** the results into a data structure

<img src="pandas-tutorial-master/img/splitApplyCombine.png">

Similar to SQL `GROUP BY`

In [ ]:
df.groupby('Sex')

In [ ]:
df.groupby("Sex").mean()

In [ ]:
df.groupby('Sex').max()

In [ ]:
def getRange(x):
    
    minVal = np.min(x.Fare)
    maxVal = np.max(x.Fare)
    
    return maxVal - minVal


df.groupby('Pclass').apply(lambda x : getRange(x))

Grouping on multiple columns

In [ ]:
df.groupby(['Sex','Pclass']).mean()

In [ ]:
df.groupby(['Sex','Pclass'])['Age'].mean()

In [ ]:
df.groupby('Sex').agg('max')

In [ ]:
df.groupby('Sex').agg({'PassengerId':'min', 'Age':'max','Fare':'sum'})

## Merge Operations

Merging with Pandas works pretty much the same as SQL. There are four merge methods:
1. Left
2. Right
3. Inner 
4. Outer

Basic syntax : pd.merge(left_dataframe, right_dataframe, left_on="some_column", right_on="some_column", how="left|right|inner|outer)`

In [ ]:
population = pd.DataFrame({'country': ['Germany', 'Belgium', 'France', 
                        'United Kingdom', 'United States'],'population': [81.3, 11.3, 64.3, 64.9, 65.9]})

countries = pd.DataFrame({'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9],
        'area': [30510, 671308, 357050, 41526, 244820],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']})

In [ ]:
population

In [ ]:
countries

In a Left Merge we are mostly concerned with data on the LEFT side but we would like to add data from 
the RIGHT side if it has some of the same countries in this case.

In [ ]:
pd.merge(left=population, right=countries, on="country", how="left")

In a Right Merge we are mostly concerned with data on the RIGHT side but we would like to add data from 
the LEFT side if it has some of the same countries in this case.

In [ ]:
pd.merge(left=population, right=countries, on="country", how="right")

With an Inner Merge, we chop up both dataframes and only glue the stuff that matches. If a country isn't in both 
dataframes, we don't keep it and we don't add NaN's. If no type of join is mentioned, then inner join is the 
default join. 

In [ ]:
pd.merge(left=population, right=countries,on ='country')

In [ ]:
pd.merge(left=population, right=countries,on ='country', how = "inner")

With an Outer Merge, we chop up both dataframes and keep everything from both sides. Then we toss in NaN's to fill
any blanks.

In [ ]:
pd.merge(left=population, right=countries,on ='country', how = "outer")

## Reading Files

In [ ]:
sales_data = pd.read_csv('pandas-tutorial-master/data/blooth_sales_data.csv')

In [ ]:
sales_data.head(5)

In [ ]:
# header = 0 denotes the first line of data. If nothing is mentioned about header, then header = 0 is default.
sales_data2 = pd.read_csv('pandas-tutorial-master/data/blooth_sales_data.csv', header = 0)

In [ ]:
sales_data2.head(5)

In [ ]:
sales_data3 = pd.read_csv('pandas-tutorial-master/data/blooth_sales_data.csv', header = None)
sales_data3.head(5)

In [ ]:
sales_data = pd.read_csv('pandas-tutorial-master/data/blooth_sales_data.csv', usecols=['name', 'birthday'])
sales_data.head(5)

In [ ]:
sales_data = pd.read_csv('pandas-tutorial-master/data/blooth_sales_data.csv', header= None, skiprows=2)
sales_data.columns= ['name', 'birthday','customer','orderadate','product','units','unitprice']
sales_data.head(2)

In [ ]:
# The date parse is US datew friendly! MM/DD/YYYY


sales_data = pd.read_csv('pandas-tutorial-master/data/blooth_sales_data.csv',parse_dates=['birthday', 'orderdate'])
sales_data.head(2)                     

In [ ]:
# To use the more common international format for sure, add 'dayfirst=True'
sales_data = pd.read_csv('pandas-tutorial-master/data/blooth_sales_data.csv',parse_dates=['birthday', 'orderdate'], dayfirst=True)
sales_data.head(2) 

In [ ]:
sales_data.dtypes

In [ ]:
sales_data['modified_orderdate'] = sales_data['orderdate'].apply(lambda x: "%d/%d/%d" % (x.day, x.month, x.year))
sales_data.head(4)

In [ ]:
sales_data.dtypes

In [ ]:
sales_data['Hour'] = sales_data['orderdate'].apply(lambda x: "%d" % (x.hour))
sales_data.head(4)

In [ ]:
sales_data["modified_orderdate"]= pd.to_datetime(sales_data["modified_orderdate"])
sales_data.head(4)
sales_data.dtypes

In [ ]:
sales_data['birth_month'] = sales_data['birthday'].dt.month
sales_data.head(4)

In [ ]:
sales_data_json = pd.read_json('pandas-tutorial-master/data/blooth_sales_data.json')
sales_data_json.head(5)

## Missing Data
How to handle missing data (NaN's)? Most common commands used are fillna and dropna. 

In [ ]:
missing_df = pd.DataFrame(np.random.randn(5, 3), index=['a', 'c', 'e', 'f', 'h'],columns=['one', 'two', 'three'])
missing_df['four'] = 'bar'
missing_df['five'] = missing_df['one'] > 0
missing_df.loc[['a','c','h'],['one','four']] = np.nan
missing_df

In [ ]:
# fillna replaces NA/NAN values with the given value in the command.
missing_df.fillna(0)

In [ ]:
missing_df['one'].fillna('missing')

Dropna is used to drop the rows or columns with NA/NAN values.
<br>
'axis' argument determines if rows or columns which contain missing values are removed.
<br>
'axis =0': Drop rows which contain missing values. 
<br>
'axis =1': Drop columns which contain missing value.
<br>


'how' argument determines if row or column is removed from DataFrame, when we have at least one NA or all NA.
<br>
‘how = any’ : If any NA values are present, drop that row or column. (default)
<br>
‘how = all’ : If all values are NA, drop that row or column.
<br>

In [ ]:
missing_df.dropna(axis=0)

In [ ]:
missing_df.dropna(axis=1)

In [ ]:
missing_df['six'] = np.nan
missing_df

In [ ]:
missing_df.dropna(axis=1, how = 'all')

In [ ]:
#dropping rows only where some columns are missing
missing_df.dropna(subset = ['one', 'two', 'four'])

In [ ]:
df.head()

# Exercises

## Titanic

In [ ]:
df.head()

Calculate the number of passengers with Pclass = 3

Compute the percentage of passengers that survived

How many children below the age of 18?

Whats the ratio of male and female passengers?

Between the two genders, whats the ratio of passengers that survived?

Create a new variable which has 0 for male and 1 for female. Name this variable **LabelEncode_Sex**

Create a variable that takes the value of 1 when Pclass is 1 and 0 otherwise. Create similar variables for when Pclass has a value of 2 and 3.

Name these variables **OHE_PClass1, OHE_PClass2, OHE_PClass3** respectively 

Calculate the mean fare for all samples with an odd index

Create a new variable which stores the last name of passengers

Calculate the number of unique families ( based on last names)

Create a variable that indicates the **size of the family** for each passenger. *Family size is the number of passengers with the same family name*

#### Fare by Cabin Index

All cabin numbers begin with a letter. We hypothesize that this first letter actually has a significance. So create a new variable that stores the first letter of the cabin variable. Call this **CabinIndex**.

NOTE : The cabin variable has missing values. Also check for the data type of the Cabin variable.

Once you have created the CabinIndex variable, calculate the mean value of fare for different levels of CabinIndex

## Sales Data

For sales_data, create a variable named mean_units which is the average of all units when the birth month lies between Feb and August.

Create a new column in sales_data titled 'order_minutes' and for each row, store the minutes from orderdate 

For sales_data dataframe, create a dataframe called 'sd_df' to store only those rows where product is 'Harry Potter book'

For sales_data, find the data of people who were born before 1980

For sales_data, find the average unitprice for products that were ordered in first week of a month

Create a column named 'count_units' in the sales_data dataframe to store the number of units sold for each product

Create a new column in sales_data and store orderdate in the format mm/dd/yyyy

## Iris Dataset

In [1]:
## Loading the dataset

from sklearn.datasets import load_iris
data = load_iris()

In [ ]:
data.data

In [ ]:
data.target

In [ ]:
data.feature_names

In [ ]:
data.target_names

### Exercises

Put together all the components of the data variable into a Pandas DataFrame. * This means putting together the feature and target variables, and adding their names as column names*

Find number of observations in the dataset which belong to class setosa and have a petal length > 3

Find the maximum and minimum values of each of features.

Find the range of value for each of the features

For each target class, find the range of values for all the features.

For each of the target classes, find the mean value of each of the independent variables. The mean values should be represented in a table.

**Do not** use for loops. This should be doable in a single line of code